In [23]:
import pandas as pd
import os

## Sanity check on complexity grouped recall/precision

In [13]:
input_csv = '/media/zhaoyang-new/workspace/KarSim/KarComparator/omkar_analyses_pipeline/builds/b14/analyses_summary/whole_analyses_table.csv'
df = pd.read_csv(input_csv)

In [16]:
df = df[df['recall'].isna()]

In [17]:
df

,file_name,cluster,n_origin_chr,origin_chr,n_path_karsim,n_path_omkar,event_chr,histories,n_events,is_event_cluster,...,n_initial_SVs,preILP_similar_SV,n_preILP_similar_SV,karsim_missed_transition_event_type,complexity score,TP,FP,FN,precision,recall


## Check if SV edge is removed between preILP and postILP

In [24]:
def read_preILP_edges(edge_file):
	sv_edges = []
	with open(edge_file) as fp_read:
		for line in fp_read:
			line = line.replace('\n', '')
			line = line.replace('(', '')
			line = line.replace(')', '')
			info = line.split(', ')
			if info[3] == "'SV'":
				sv_edges.append({'in-node': info[0],
								 'out-node': info[1]})
	return sv_edges

In [47]:
def read_postILP_edges(edge_file_folder):
	sv_edges = []
	for filename in os.listdir(edge_file_folder):
		file_path = os.path.join(edge_file_folder, filename)
		with open(file_path) as fp_read:
			for line in fp_read:
				line = line.replace('\n', '')
				line = line.replace('(', '')
				line = line.replace(')', '')
				info = line.split(', ')
				if info[3] in ["'SV'", "'D'"]:
					sv_edges.append({'in-node': info[0],
									 'out-node': info[1],
									 'type': info[3]})
	return sv_edges

In [48]:
omkar_output_folder = '/media/zhaoyang-new/workspace/KarSim/KarComparator/omkar_analyses_pipeline/builds/b14/omkar_output/'

In [49]:
def check_removed_edge(preILP, postILP):
	missed_edge = []
	for pre_edge in preILP:
		edge_found = False
		for post_edge in postILP:
			if pre_edge['in-node'] == post_edge['in-node'] and pre_edge['out-node'] == post_edge['out-node']:
				edge_found = True
				break
			elif pre_edge['in-node'] == post_edge['out-node'] and pre_edge['out-node'] == post_edge['in-node']:
				# undirected graph
				edge_found = True
				break
		if not edge_found:
			missed_edge.append(pre_edge)
	return missed_edge

In [54]:
def check_if_SV_removed_added_as_dummy(preILP, postILP):
	SV_missed_edges = []
	for pre_edge in preILP:
		edge_found = False
		for post_edge in postILP:
			if post_edge['type'] == "'SV'":
				if pre_edge['in-node'] == post_edge['in-node'] and pre_edge['out-node'] == post_edge['out-node']:
					edge_found = True
					break
				elif pre_edge['in-node'] == post_edge['out-node'] and pre_edge['out-node'] == post_edge['in-node']:
					# undirected graph
					edge_found = True
					break
		if not edge_found:
			SV_missed_edges.append(pre_edge)
	dummy_added_back = []
	for pre_edge in SV_missed_edges:
		edge_found = False
		for post_edge in postILP:
			if post_edge['type'] == "'D'":
				if pre_edge['in-node'] == post_edge['in-node'] and pre_edge['out-node'] == post_edge['out-node']:
					edge_found = True
					break
				elif pre_edge['in-node'] == post_edge['out-node'] and pre_edge['out-node'] == post_edge['in-node']:
					# undirected graph
					edge_found = True
					break
		if edge_found:
			dummy_added_back.append(pre_edge)
	return dummy_added_back

In [55]:
total_sv_edges_received = 0
total_sv_edeges_removed = 0
total_dummies_added_back = 0
for case_folder in os.listdir(omkar_output_folder):
	preILP_file = os.path.join(omkar_output_folder, case_folder, '{}.preILP_edges.txt'.format(case_folder))
	postILP_file_folder = os.path.join(omkar_output_folder, case_folder, 'all_edges_with_dummies')
	preILP_edges = read_preILP_edges(preILP_file)
	postILP_edges = read_postILP_edges(postILP_file_folder)
	missed_edges = check_removed_edge(preILP_edges, postILP_edges)
	dummy_added_back = check_if_SV_removed_added_as_dummy(preILP_edges, postILP_edges)
	total_sv_edges_received += len(preILP_edges)
	total_sv_edeges_removed += len(missed_edges)
	total_dummies_added_back += len(dummy_added_back)

print(total_sv_edges_received)
print(total_sv_edeges_removed)
print(total_dummies_added_back)

721
110
0
